In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True


seed_everything(86)

In [ ]:
train_df = pd.read_csv("sentences.csv")
test_df = pd.read_csv("test sentences.csv")

In [ ]:
device = torch.device("cpu")
labels = ["enroll", "search", "inform"]
label_encoder = LabelEncoder()
label_encoder.fit_transform(labels)

train_sentences = train_df["Sentence"].tolist()
test_sentences = test_df["Sentence"].tolist()
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)

train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

train_padded = pad_sequences(train_sequences, maxlen=30, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=30, padding="post", truncating="post")

train_inputs = torch.tensor(train_padded)
test_inputs = torch.tensor(test_padded)

train_labels = label_encoder.transform(train_df["Intent"].values)
test_labels = label_encoder.transform(test_df["Intent"].values)

train_targets = torch.tensor(train_labels)
test_targets = torch.tensor(test_labels)

train_dataset = TensorDataset(train_inputs, train_targets)
test_dataset = TensorDataset(test_inputs, test_targets)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

class IntentClassifier(nn.Module):
    def __init__(self):
        super(IntentClassifier, self).__init__()
        self.embedding = nn.Embedding(1000, 16)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 30, 6)
        self.fc2 = nn.Linear(6, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        embedded = self.embedding(x)
        flatten = self.flatten(embedded)
        out = self.relu(self.fc1(flatten))
        out = self.fc2(out)
        return out

In [ ]:
def train(model, criterion, optimizer, loader):
    model.train()
    size = len(loader.dataset)
    num_batches = len(loader)
    train_loss, correct = 0, 0

    for inputs, targets in loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (outputs.argmax(1) == targets).sum().item()

    train_loss /= num_batches
    correct /= size
    print(f"loss: {train_loss:.4f}   accuracy: {correct:.4f}   ", end="")

def eval(model, criterion, loader):
    model.eval()
    size = len(loader.dataset)
    num_batches = len(loader)
    val_loss, correct = 0, 0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            val_loss += loss.item()
            correct += (outputs.argmax(1) == targets).sum().item()

    val_loss /= num_batches
    correct /= size
    print(f"val_loss: {val_loss:.4f}   val_accuracy: {correct:.4f}")

    return correct

In [ ]:
num_epochs = 100

model = IntentClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.0029)

best_acc = 0
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}   ", end="")

    train(model, criterion, optimizer, train_loader)
    val_acc = eval(model, criterion, test_loader)

    if val_acc > best_acc:
        torch.save(model.state_dict(), f"intent.pth")
        best_acc = val_acc

print(f"Average accuracy: {best_acc:.4f}")

Epoch 1/100   loss: 0.7255   accuracy: 0.6557   val_loss: 0.5930   val_accuracy: 0.8585
Epoch 2/100   loss: 0.3338   accuracy: 0.9505   val_loss: 0.3116   val_accuracy: 0.9340
Epoch 3/100   loss: 0.1476   accuracy: 0.9929   val_loss: 0.2435   val_accuracy: 0.9057
Epoch 4/100   loss: 0.0819   accuracy: 1.0000   val_loss: 0.1743   val_accuracy: 0.9434
Epoch 5/100   loss: 0.0514   accuracy: 1.0000   val_loss: 0.1559   val_accuracy: 0.9528
Epoch 6/100   loss: 0.0342   accuracy: 1.0000   val_loss: 0.1453   val_accuracy: 0.9528
Epoch 7/100   loss: 0.0237   accuracy: 1.0000   val_loss: 0.1425   val_accuracy: 0.9528
Epoch 8/100   loss: 0.0172   accuracy: 1.0000   val_loss: 0.1411   val_accuracy: 0.9528
Epoch 9/100   loss: 0.0129   accuracy: 1.0000   val_loss: 0.1326   val_accuracy: 0.9528
Epoch 10/100   loss: 0.0097   accuracy: 1.0000   val_loss: 0.1241   val_accuracy: 0.9528
Epoch 11/100   loss: 0.0075   accuracy: 1.0000   val_loss: 0.1184   val_accuracy: 0.9528
Epoch 12/100   loss: 0.0058   

In [ ]:
model = IntentClassifier().to(device)
model.load_state_dict(torch.load(f"intent.pth"))
model.eval()

IntentClassifier(
  (embedding): Embedding(1000, 16)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=480, out_features=6, bias=True)
  (fc2): Linear(in_features=6, out_features=3, bias=True)
  (relu): ReLU()
)

In [ ]:
from sklearn.metrics import classification_report

def test(loader):
    predicts = []
    true_labels = []

    with torch.no_grad():
        for inputs, targets in loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            _, pred = torch.max(outputs, 1)
            predicts.extend(pred)
            true_labels.extend(targets)

    predicts = torch.stack(predicts).cpu()
    true_labels = torch.stack(true_labels).cpu()
    print(classification_report(true_labels, predicts, digits=4))

In [ ]:
test(test_loader)

              precision    recall  f1-score   support

           0     1.0000    0.9608    0.9800        51
           1     1.0000    1.0000    1.0000        27
           2     0.9333    1.0000    0.9655        28

    accuracy                         0.9811       106
   macro avg     0.9778    0.9869    0.9818       106
weighted avg     0.9824    0.9811    0.9813       106



In [ ]:
print(label_encoder.inverse_transform([0, 1, 2]))

['enroll' 'inform' 'search']


In [ ]:
def identify_intent(sentence):
    sequence = tokenizer.texts_to_sequences([sentence])
    padded = pad_sequences(sequence, maxlen=30, padding="post", truncating="post")
    inputs = torch.tensor(padded).to(device)

    outputs = model(inputs)
    pred = torch.argmax(outputs, 1).item()
    intent = label_encoder.inverse_transform([pred])[0]

    return intent

In [ ]:
print(identify_intent("Enroll me in 6.006 for winter"))
print(identify_intent("List me some Physics course"))
print(identify_intent("could you list several physics courses"))

enroll
search
search
